# Part II: Get Neighborhoods coordinates
## Part I: Web Scraping Wikipedia Tables using BeautifulSoup and Python

## IBM Data Science Specialization
##### Project Capstone. 
### Foursquare. Toronto Neighborhoods Segmentation and Classsification
**Article in Medium: Introduction to Web Scraping with BeautifulSoup**
https://towardsdatascience.com/introduction-to-web-scraping-with-beautifulsoup-e87a06c2b857

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Library `request` allows you to send organic, grass-fed HTTP/1.1 requests, without the need for manual labor.
#!conda install -c anaconda requests --yes

In [3]:
# importing libraries
from bs4 import BeautifulSoup
import urllib.request
import re

### Define url. Wikipedia: "List of postal codes of Canada: M"

In [4]:
# Define url. Wikipedia: "List of postal codes of Canada: M" 
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [5]:
# Connect to the website using urllib .
try:
    req = urllib.request.urlopen(url) # conntect to website
except:
    print("An error occured.")

In [6]:
article = req.read().decode()
#The ISO 3166-1 alpha-2 contains this information in an HTML table 
# which can be scraped quite easily as follows.

with open('ISO_3166-1_alpha-2.html', 'w') as fo:
    fo.write(article)

### Load and parse the Wikipedia page
Then load it and parse it with Beautiful Soup. Extract all the `<table>` tags and search for the one with the headings corresponding to the data we want.

In [7]:
from bs4 import BeautifulSoup

# Load article, turn into soup and get the <table>s.
article = open('ISO_3166-1_alpha-2.html').read()
soup = BeautifulSoup(article, 'html.parser')
table_str = soup.find_all('table', class_='sortable')
len(table_str)

1

The variable **`table_str`** looks like:

`
[<table class="wikitable sortable">
 <tbody><tr>
 <th>Postcode</th>
 <th>Borough</th>
 <th>Neighbourhood
 </th></tr>
 <tr>
 <td>M1A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>
 ...
 <tr>
 <td>M3B</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td>Don Mills North
 </td></tr>
 ...
 <tr>
 <td>M5A</td>
 <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
 <td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
 </td></tr>
 ...
 ...
 ...
 <tr>
 <td>M9Z</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>
 </tbody></table>]`

### Find specific elements in the page, i.e. the  contain of the cell's table.
The created BeautifulSoup object can now be used to find elements in the HTML. When we inspected the website we saw that every list item in the content section has a class that starts with `th` (`</th>`) and we can us BeautifulSoup’s `find_all` method to find all list items with that class.

In [8]:
for table in table_str:
    ths = table.find_all('th')
    heading = [th.text.strip() for th in ths]

print('Data Frame columns: ', heading)

Data Frame columns:  ['Postcode', 'Borough', 'Neighbourhood']


When we inspected the website we saw that every new entry or row of the table starts with `tr` and after in the row the elements **Postcode**, **Borough** and **Neighbourhood** starts with `td` (`</td>`) and we can us BeautifulSoup’s `find_all` method to find all list items with that class.

In [9]:
for table in table_str:
    tds = table.find_all('td')
    body = [th.text.strip() for th in tds]

#Display the first 6 elements of body
body[0:7]

['M1A',
 'Not assigned',
 'Not assigned',
 'M2A',
 'Not assigned',
 'Not assigned',
 'M3A']

### Convert variable `body` into a Pandas DataFrame: `my_table`

Assign column names and Define DataFrame contain

In [10]:
my_table=pd.DataFrame([], columns=heading)
my_table['Postcode']=body[0::3]
my_table['Borough']=body[1::3]
my_table['Neighbourhood']=body[2::3]

Display 6 first rows. Sort table ordered by `Borough`

In [11]:
my_table.sort_values(by='Borough').head(6)

,Postcode,Borough,Neighbourhood
211,M4V,Central Toronto,Deer Park
158,M5P,Central Toronto,Forest Hill West
197,M4T,Central Toronto,Summerhill East
168,M4R,Central Toronto,North Toronto West
157,M5P,Central Toronto,Forest Hill North
156,M4P,Central Toronto,Davisville North


### Nr. of unique values and values of `Borough`
We can see that `Borough` contains the value `'Not assigned'`

In [12]:
print('*** Unique values (Borohoods):\n',  my_table.Borough.unique())
print('\n*** Nr of Unique values(Borohoods): ', my_table.Borough.nunique())

*** Unique values (Borohoods):
 ['Not assigned' 'North York' 'Downtown Toronto' "Queen's Park" 'Etobicoke'
 'Scarborough' 'East York' 'York' 'East Toronto' 'West Toronto'
 'Central Toronto' 'Mississauga']

*** Nr of Unique values(Borohoods):  12


### Reject rows  which Borougs =  `Not assigned`

In [13]:
# Slice my_table and discard all columns which 'Borough' is equal to 'Not assigned'
my_table=my_table[my_table.Borough != 'Not assigned']
bor_names=my_table.Borough.unique()
print('*** Nr of Unique values(Borohoods): ', my_table.Borough.nunique())
print('\n*** Unique values (Borohoods): \n','\n', my_table.Borough.unique())

*** Nr of Unique values(Borohoods):  11

*** Unique values (Borohoods): 
 
 ['North York' 'Downtown Toronto' "Queen's Park" 'Etobicoke' 'Scarborough'
 'East York' 'York' 'East Toronto' 'West Toronto' 'Central Toronto'
 'Mississauga']


Show number of Neighbourhoods and 5 first values.

### Nr. of unique values and values of `Neighbourhoods`

In [14]:

unique_neig=my_table.Neighbourhood.unique()
print('\n*** Nr of Unique values(Neighbourhoods): ', my_table.Neighbourhood.nunique())
print('*** 5 first Unique values (Neighbourhoods):\n', my_table.Neighbourhood.unique()[0:5])


*** Nr of Unique values(Neighbourhoods):  209
*** 5 first Unique values (Neighbourhoods):
 ['Parkwoods' 'Victoria Village' 'Harbourfront' 'Regent Park'
 'Lawrence Heights']


### Replace  ` Neighbourhood = 'Not assigned'` with its `Borough` value

In [15]:
# Replace columns which 'Neighbourhood' is equal to 'Not assigned'
vect=list(my_table[my_table.Neighbourhood == 'Not assigned'].Borough)
my_table=my_table.replace('Not assigned',vect[0])

In [16]:
my_table.head(8)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue


Postcode `M7A` (row_index=8) => The Neighbourhood is `Queen's Park` now. Same as its Borough

### Create dictionary `my_Dict` with all `'Postcode', 'Borough','Neighborhood'`

In [17]:
my_Dict={}

for pc in my_table.Postcode.unique():
    nr=my_table[my_table.Postcode == pc].Neighbourhood.nunique()
    bor=my_table[my_table.Postcode == pc].Borough.unique()
    
    if nr == 1:
        ne=my_table[my_table.Postcode == pc].Neighbourhood
        my_Dict[pc]={'Postcode': pc, 'Borough': bor[0],'Neighborhood':list(ne)}
    if nr>1:
        w=[]
        for z in my_table[my_table.Postcode == pc].Neighbourhood:
            w.append(z)
        my_Dict[pc]={'Postcode': pc,'Borough': bor[0],'Neighborhood':list(w)}    

#### Calling elements of  `my_Dict`
Examples:

In [18]:
print(my_Dict['M6A'])
print(my_Dict['M6A']['Neighborhood'])

{'Postcode': 'M6A', 'Borough': 'North York', 'Neighborhood': ['Lawrence Heights', 'Lawrence Manor']}
['Lawrence Heights', 'Lawrence Manor']


## Create pandas DataFrame table `df` from dictionaty `my_Dict`

Get a string representation of a python list, (do not show brackets) with `.join()`

In [19]:
df=pd.DataFrame(my_Dict).T
df = df[['Postcode', 'Borough', 'Neighborhood']]
df=df.reset_index(drop=True)
my_table=df
df['Neighborhood'] = df.Neighborhood.apply(', '.join)
my_table.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


# Solution `df`:

Show first 15 rows

In [20]:
df.head(5)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [21]:
df.tail()

,Postcode,Borough,Neighborhood
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."
102,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."


# Part II: Get neighborhoods coordinates

In [22]:
#!conda install -c conda-forge geocoder --yes

In [23]:
import geocoder # import geocoder

In [24]:
import geocoder

lat_lng_coords = None

#while(lat_lng_coords is None):
#    g = geocoder.google('Mountain View, CA')
#    lat_lng_coords = g.latlng
#print(g.latlng)

In [25]:
#import geocoder # import geocoder

#postal_code='MC4'

# initialize your variable to None
#lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
#    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

In [26]:
import pandas as pd
df_geo=pd.read_csv('input/Geospatial_Coordinates.csv')

In [27]:
df_geo.shape

(103, 3)

In [28]:
df_geo.tail()

,Postal Code,Latitude,Longitude
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437
102,M9W,43.706748,-79.594054


In [29]:
df_geo.columns=['Postcode', 'latitude', 'longitude']
df_geo.head(2)

,Postcode,latitude,longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497


In [30]:
print(df.shape)
df.head(2)

(103, 3)


,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village


In [31]:
df_toronto=pd.merge(df, df_geo, on='Postcode', how='outer')
print(df_toronto.shape)
df_toronto.tail(4)

(103, 5)


,Postcode,Borough,Neighborhood,latitude,longitude
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509
102,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen...",43.628841,-79.520999


In [32]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [33]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto['Borough'].unique()),
        df_toronto.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [34]:
import folium
print('Folium installed and imported!')

Folium installed and imported!


In [35]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [36]:
latitude=43.653963
longitude=-79.387207

In [37]:
# create map and display it
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# display the map of Toronto
#map_toronto

In [38]:
# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['latitude'], df_toronto['longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [39]:
CLIENT_ID = 'M2X2X0DLGLZOPJ3RDWZN5J2VN0ZBNPQULREX1PK4F4NYRISL' # your Foursquare ID
CLIENT_SECRET = 'JV2PAYVQNRIDJ3XRJGO32WEOHXCUVWAPT3Y43JHYHOMAO3PL' # your Foursquare Secret
VERSION = '20200604'
LIMIT = 300
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: M2X2X0DLGLZOPJ3RDWZN5J2VN0ZBNPQULREX1PK4F4NYRISL
CLIENT_SECRET:JV2PAYVQNRIDJ3XRJGO32WEOHXCUVWAPT3Y43JHYHOMAO3PL


# Explore venues in Toronto
### Let's create the  function `getNearbyVenues` to rretrieve venues for all the neighborhoods in Toronto

In [40]:
#Define radius and limit
radius = 500
LIMIT=300

In [41]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *`toronto_venues`*.
### `toronto_venues`

In [42]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['latitude'],
                                   longitudes=df_toronto['longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

#### Check `toronto_venues` DataFrame. Get dimensions of DataFrame

In [43]:
print(toronto_venues.shape)
toronto_venues.sort_values(by='Neighborhood').head(2)

(2231, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
717,"Adelaide, King, Richmond",43.650571,-79.384568,LUSH,43.653557,-79.380400,Cosmetics Shop
684,"Adelaide, King, Richmond",43.650571,-79.384568,Kupfert & Kim (First Canadian Place),43.648547,-79.381624,Gluten-free Restaurant


In [44]:
print('Nr. unique Neighborhoods in toronto_venues:', toronto_venues.Neighborhood.nunique())

Nr. unique Neighborhoods in toronto_venues: 100


There are only **100 neighborhoods in `toronto_venues`** that have venues. However **`df_toronto`contains 103 neighborhoods**. 
* **Problem!!!!** We need to find out which neighborhoods do not appear in `toronto_venues`

### Let's check how many venues were returned for each neighborhood

In [45]:
#print(toronto_venues.groupby('Neighborhood').Venue.count())
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",9,9,9,9,9,9
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Downsview North, Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,56,56,56,56,56,56


### Find the neighborhoods in `df_toronto` that do not appear in `toronto_venues`

In [46]:
df_toronto.Neighborhood[np.logical_not(df_toronto.Neighborhood.isin(toronto_venues.Neighborhood))]

5            Islington Avenue
52    Newtonbrook, Willowdale
95                Upper Rouge
Name: Neighborhood, dtype: object

#### Redefine `df_toronto` remove the rows with index `5,52,95`and reset the dataframe index
After this, the new  `df_toronto` should have 100 rows now

In [47]:
df_toronto=df_toronto[df_toronto.Neighborhood.isin(toronto_venues.Neighborhood)].reset_index()
print('Shape of new toronto_df:', df_toronto.shape)
df_toronto.tail(4)

Shape of new toronto_df: (100, 6)


,index,Postcode,Borough,Neighborhood,latitude,longitude
96,99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
97,100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
98,101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509
99,102,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen...",43.628841,-79.520999


#### Let's find out how many unique categories can be curated from all the returned venues

In [48]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 274 uniques categories.


## 3. Analyze Each Neighborhood

In [49]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_onehot.head(2)

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [50]:
i=-1
for col in toronto_onehot.columns:
    i=i+1
    if col == 'Neighborhood':         
        neig_col_nr=i
        print(i,col,neig_col_nr)
print('Nr of columns in toronto_onehot',len(toronto_onehot.columns))

193 Neighborhood 193
Nr of columns in toronto_onehot 274


In [51]:
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[neig_col_nr]] + list(toronto_onehot.columns[0:neig_col_nr])+list(toronto_onehot.columns[neig_col_nr+1:])
i=-1
for col in fixed_columns:
    i=i+1
    if col == 'Neighborhood':         
        neig_col_nr=i
        print('*******',i,col,neig_col_nr)
print('Nr of elements in fixed_columns',len(fixed_columns))


******* 0 Neighborhood 0
Nr of elements in fixed_columns 274


In [52]:
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head(2)

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [53]:
toronto_onehot.shape

(2231, 274)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [54]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [55]:
toronto_grouped.head(5)

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03,0.0,0.0,0.01,0.

In [56]:
print(toronto_grouped.shape)

(100, 274)


Observe that hte number of neighborhoods in `toronto_grouped` is 100 in agreement with the number of neighborhoods in `df_toronto`.

#### Let's print each neighborhood along with the top 5 most common venues

In [57]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("---- "+hood+" ----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Adelaide, King, Richmond ----
             venue  freq
0      Coffee Shop  0.08
1             Café  0.05
2              Bar  0.04
3  Thai Restaurant  0.04
4       Steakhouse  0.04


---- Agincourt ----
            venue  freq
0    Skating Rink  0.25
1          Lounge  0.25
2  Clothing Store  0.25
3  Breakfast Spot  0.25
4     Men's Store  0.00


---- Agincourt North, L'Amoreaux East, Milliken, Steeles East ----
                 venue  freq
0           Playground   0.5
1                 Park   0.5
2    Accessories Store   0.0
3   Mexican Restaurant   0.0
4  Monument / Landmark   0.0


---- Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown ----
                 venue  freq
0        Grocery Store  0.22
1          Pizza Place  0.11
2  Fried Chicken Joint  0.11
3          Coffee Shop  0.11
4           Beer Store  0.11


---- Alderwood, Long Branch ----
            venue  freq
0     Pizza Place  0.22
1        Pharmacy  0.11
2  

                        venue  freq
0  Construction & Landscaping  0.25
1                        Park  0.25
2            Basketball Court  0.25
3                      Bakery  0.25
4           Accessories Store  0.00


---- East Birchmount Park, Ionview, Kennedy Park ----
                venue  freq
0         Coffee Shop  0.25
1          Hobby Shop  0.25
2  Chinese Restaurant  0.25
3    Department Store  0.25
4   Mobile Phone Shop  0.00


---- East Toronto ----
                venue  freq
0                Park   0.4
1         Pizza Place   0.2
2   Convenience Store   0.2
3         Coffee Shop   0.2
4  Miscellaneous Shop   0.0


---- Emery, Humberlea ----
                        venue  freq
0  Construction & Landscaping   0.5
1              Baseball Field   0.5
2           Accessories Store   0.0
3          Miscellaneous Shop   0.0
4                       Motel   0.0


---- Fairview, Henry Farm, Oriole ----
                  venue  freq
0        Clothing Store  0.16
1  Fast Food Restaura

                venue  freq
0                Café  0.08
1         Pizza Place  0.08
2         Coffee Shop  0.08
3  Italian Restaurant  0.05
4    Sushi Restaurant  0.05


---- Ryerson, Garden District ----
                       venue  freq
0                Coffee Shop  0.09
1             Clothing Store  0.06
2             Cosmetics Shop  0.04
3                       Café  0.03
4  Middle Eastern Restaurant  0.03


---- Scarborough Village ----
                       venue  freq
0                 Playground   0.5
1          Convenience Store   0.5
2          Accessories Store   0.0
3  Middle Eastern Restaurant   0.0
4                      Motel   0.0


---- Silver Hills, York Mills ----
                 venue  freq
0            Cafeteria   1.0
1    Accessories Store   0.0
2   Miscellaneous Shop   0.0
3                Motel   0.0
4  Monument / Landmark   0.0


---- St. James Town ----
                venue  freq
0         Coffee Shop  0.06
1                Café  0.05
2          Restaurant

#### Let's put that into a *pandas* dataframe

First, let's write a **function** `return_most_common_venues` to sort the venues in descending order.

In [58]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [59]:
num_top_venues = 40

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(3)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Gym,Asian Restaurant,Hotel,Restaurant,American Restaurant,Breakfast Spot,Burger Joint,Sushi Restaurant,Salad Place,Bakery,Bookstore,Pizza Place,Gastropub,Cosmetics Shop,Concert Hall,Ice Cream Shop,Brazilian Restaurant,Electronics Store,Mediterranean Restaurant,Smoke Shop,Seafood Restaurant,Noodle House,Salon / Barbershop,Building,Office,Burrito Place,Indian Restaurant,Record Shop,Latin American Restaurant,Italian Restaurant,Poke Place,Plaza,Japanese Restaurant,Opera House,Clothing Store
1,Agincourt,Breakfast Spot,Lounge,Skating Rink,Clothing Store,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Electronics Store,Eastern European Restaurant,Diner,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Discount Store,Yoga Studio,Farmers Market,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop,Dim Sum Restaurant,Fast Food Restaurant,Colombian Restaurant,German Restaurant,Garden,Garden Center
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Yoga Studio,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Dim Sum Restaurant,Department Store,Farmers Market,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop,Comfort Food Restaurant,Falafel Restaurant,Fast Food Restaurant,College Stadium,German Restaurant,Garden,Garden Center,Gastropub


In [60]:
neighborhoods_venues_sorted.tail(3)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue
97,"Woodbine Gardens, Parkview Hill",Fast Food Restaurant,Pizza Place,Gastropub,Gym / Fitness Center,Pharmacy,Café,Bank,Intersection,Athletics & Sports,Pet Store,General Entertainment,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Dive Bar,Golf Course,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Dance Studio,Gay Bar,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Gourmet Shop,Concert Hall,Greek Restaurant,Grocery Store,Comic Shop
98,Woodbine Heights,Park,Pharmacy,Skating Rink,Video Store,Bus Stop,Beer Store,Cosmetics Shop,Athletics & Sports,Dance Studio,Curling Ice,Donut Shop,Dog Run,Doner Restaurant,Eastern European Restaurant,Drugstore,Dumpling Restaurant,Discount Store,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Dive Bar,Department Store,Diner,Dim Sum Restaurant,Dessert Shop,Falafel Restaurant,Deli / Bodega,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop,Comfort Food Restaurant,Event Space,Yoga Studio,Farmers Market,General Travel
99,York Mills West,Park,Bank,Convenience Store,Flower Shop,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Electronics Store,Eastern European Restaurant,Discount Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Dive Bar,Dim Sum Restaurant,Diner,Fast Food Restaurant,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Construction & Landscaping,Concert Hall,Comic Shop,Farmers Market,Filipino Restaurant,Field,Garden,Gym / Fitness Center,Gym


## 4. Cluster Neighborhoods

### Run *k*-means to cluster the neighborhood into 5 clusters.

In [61]:
# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 3, 0, 0, 0, 0, 0], dtype=int32)

In [62]:
zz=pd.DataFrame(kmeans.labels_)
zz.columns=['labels']
print(toronto_grouped.shape)
print(toronto_grouped_clustering.shape)
print(zz.shape)
zz.labels.unique()

(100, 274)
(100, 273)
(100, 1)


array([0, 1, 3, 4, 5, 2, 6])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [63]:
# add clustering labels
neighborhoods_venues_sorted['Cluster Labels']=kmeans.labels_
neighborhoods_venues_sorted.head(3)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,Cluster Labels
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Gym,Asian Restaurant,Hotel,Restaurant,American Restaurant,Breakfast Spot,Burger Joint,Sushi Restaurant,Salad Place,Bakery,Bookstore,Pizza Place,Gastropub,Cosmetics Shop,Concert Hall,Ice Cream Shop,Brazilian Restaurant,Electronics Store,Mediterranean Restaurant,Smoke Shop,Seafood Restaurant,Noodle House,Salon / Barbershop,Building,Office,Burrito Place,Indian Restaurant,Record Shop,Latin American Restaurant,Italian Restaurant,Poke Place,Plaza,Japanese Restaurant,Opera House,Clothing Store,0
1,Agincourt,Breakfast Spot,Lounge,Skating Rink,Clothing Store,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Electronics Store,Eastern European Restaurant,Diner,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Discount Store,Yoga Studio,Farmers Market,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop,Dim Sum Restaurant,Fast Food Restaurant,Colombian Restaurant,German Restaurant,Garden,Garden Center,0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Yoga Studio,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Dim Sum Restaurant,Department Store,Farmers Market,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop,Comfort Food Restaurant,Falafel Restaurant,Fast Food Restaurant,College Stadium,German Restaurant,Garden,Garden Center,Gastropub,1


In [64]:
toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,index,Postcode,Borough,Neighborhood,latitude,longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,Cluster Labels
0,0,M3A,North York,Parkwoods,43.753259,-79.329656,Fast Food Restaurant,Park,Food & Drink Shop,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio,Diner,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Eastern European Restaurant,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop,Farmers Market,Field,Colombian Restaurant,Garden,Gym / Fitness Center,Gym,Grocery Store,1
1,1,M4A,North York,Victoria Village,43.725882,-79.315572,Portuguese Restaurant,Coffee Shop,Hockey Arena,Intersection,Ethiopian Restaurant,Empanada Restaurant,Event Space,Electronics Store,Falafel Restaurant,Eastern European Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Dive Bar,Discount Store,Diner,Yoga Studio,Dessert Shop,Fast Food Restaurant,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop,Farmers Market,Field,Colombian Restaurant,Gaming Cafe,Gym,0
2,2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,Coffee Shop,Park,Bakery,Café,Mexican Restaurant,Pub,Breakfast Spot,Electronics Store,Farmers Market,Spa,Chocolate Shop,Italian Restaurant,Beer Store,Brewery,Dessert Shop,Ice Cream Shop,French Restaurant,Restaurant,Bank,Performing Arts Venue,Shoe Store,Event Space,Theater,Cosmetics Shop,Art Gallery,Historic Site,Antique Shop,Health Food Store,Gym / Fitness Center,Dive Bar,Drugstore,German Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,General Entertainment,Discount Store,Diner,Gift Shop,0
3,3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,Clothing Store,Furniture / Home Store,Coffee Shop,Fraternity House,Miscellaneous Shop,Arts & Crafts Store,Accessories Store,Vietnamese Restaurant,Event Space,Boutique,Women's Store,Empanada Restaurant,Electronics Store,Ethiopian Restaurant,Eastern European Restaurant,Discount Store,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Dive Bar,Dim Sum Restaurant,Diner,Farmers Market,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Falafel Restaurant,Filipino Restaurant,Fast Food Restaurant,0
4,4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,Coffee Shop,Diner,Gym,Park,Burrito Place,Italian Restaurant,Spa,Japanese Restaurant,Fast Food Restaurant,Seafood Restaurant,Sandwich Place,Burger Joint,Yoga Studio,Café,Sushi Restaurant,Portuguese Restaurant,Mexican Restaurant,Chinese Restaurant,Creperie,College Auditorium,Bar,Smoothie Shop,Arts & Crafts Store,Theater,Wings Joint,Hobby Shop,Vegetarian / Vegan Restaur

In [65]:
print(toronto_merged.shape)
print(neighborhoods_venues_sorted.shape)
toronto_merged.Neighborhood==neighborhoods_venues_sorted.Neighborhood
toronto_merged.head(3)

(100, 47)
(100, 42)


,index,Postcode,Borough,Neighborhood,latitude,longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,Cluster Labels
0,0,M3A,North York,Parkwoods,43.753259,-79.329656,Fast Food Restaurant,Park,Food & Drink Shop,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio,Diner,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Eastern European Restaurant,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop,Farmers Market,Field,Colombian Restaurant,Garden,Gym / Fitness Center,Gym,Grocery Store,1
1,1,M4A,North York,Victoria Village,43.725882,-79.315572,Portuguese Restaurant,Coffee Shop,Hockey Arena,Intersection,Ethiopian Restaurant,Empanada Restaurant,Event Space,Electronics Store,Falafel Restaurant,Eastern European Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Dive Bar,Discount Store,Diner,Yoga Studio,Dessert Shop,Fast Food Restaurant,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop,Farmers Market,Field,Colombian Restaurant,Gaming Cafe,Gym,0
2,2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,Coffee Shop,Park,Bakery,Café,Mexican Restaurant,Pub,Breakfast Spot,Electronics Store,Farmers Market,Spa,Chocolate Shop,Italian Restaurant,Beer Store,Brewery,Dessert Shop,Ice Cream Shop,French Restaurant,Restaurant,Bank,Performing Arts Venue,Shoe Store,Event Space,Theater,Cosmetics Shop,Art Gallery,Historic Site,Antique Shop,Health Food Store,Gym / Fitness Center,Dive Bar,Drugstore,German Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,General Entertainment,Discount Store,Diner,Gift Shop,0


### Visualize the resulting clusters

In [66]:
toronto_merged['Cluster Labels'].unique()

array([1, 0, 4, 3, 6, 5, 2])

In [67]:
print(toronto_merged.shape)
toronto_merged.head(20)

(100, 47)


,index,Postcode,Borough,Neighborhood,latitude,longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,Cluster Labels
0,0,M3A,North York,Parkwoods,43.753259,-79.329656,Fast Food Restaurant,Park,Food & Drink Shop,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio,Diner,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Eastern European Restaurant,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop,Farmers Market,Field,Colombian Restaurant,Garden,Gym / Fitness Center,Gym,Grocery Store,1
1,1,M4A,North York,Victoria Village,43.725882,-79.315572,Portuguese Restaurant,Coffee Shop,Hockey Arena,Intersection,Ethiopian Restaurant,Empanada Restaurant,Event Space,Electronics Store,Falafel Restaurant,Eastern European Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Dive Bar,Discount Store,Diner,Yoga Studio,Dessert Shop,Fast Food Restaurant,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop,Farmers Market,Field,Colombian Restaurant,Gaming Cafe,Gym,0
2,2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,Coffee Shop,Park,Bakery,Café,Mexican Restaurant,Pub,Breakfast Spot,Electronics Store,Farmers Market,Spa,Chocolate Shop,Italian Restaurant,Beer Store,Brewery,Dessert Shop,Ice Cream Shop,French Restaurant,Restaurant,Bank,Performing Arts Venue,Shoe Store,Event Space,Theater,Cosmetics Shop,Art Gallery,Historic Site,Antique Shop,Health Food Store,Gym / Fitness Center,Dive Bar,Drugstore,German Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,General Entertainment,Discount Store,Diner,Gift Shop,0
3,3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,Clothing Store,Furniture / Home Store,Coffee Shop,Fraternity House,Miscellaneous Shop,Arts & Crafts Store,Accessories Store,Vietnamese Restaurant,Event Space,Boutique,Women's Store,Empanada Restaurant,Electronics Store,Ethiopian Restaurant,Eastern European Restaurant,Discount Store,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Dive Bar,Dim Sum Restaurant,Diner,Farmers Market,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Falafel Restaurant,Filipino Restaurant,Fast Food Restaurant,0
4,4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,Coffee Shop,Diner,Gym,Park,Burrito Place,Italian Restaurant,Spa,Japanese Restaurant,Fast Food Restaurant,Seafood Restaurant,Sandwich Place,Burger Joint,Yoga Studio,Café,Sushi Restaurant,Portuguese Restaurant,Mexican Restaurant,Chinese Restaurant,Creperie,College Auditorium,Bar,Smoothie Shop,Arts & Crafts Store,Theater,Wings Joint,Hobby Shop,Vegetarian / Vegan Restaur

In [68]:
latitude=43.850882
longitude=-79.315572

In [69]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10.2)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [70]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['latitude'], toronto_merged['longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    cluster=int(cluster)
    #print('cluster', cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster. 

#### Cluster 0

In [71]:
c0=toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
print('cluster_0. Nr of Neighborhoods', c0.shape[0])
c0

cluster_0. Nr of Neighborhoods 82


,Postcode,longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,Cluster Labels
1,M4A,-79.315572,Portuguese Restaurant,Coffee Shop,Hockey Arena,Intersection,Ethiopian Restaurant,Empanada Restaurant,Event Space,Electronics Store,Falafel Restaurant,Eastern European Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Dive Bar,Discount Store,Diner,Yoga Studio,Dessert Shop,Fast Food Restaurant,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop,Farmers Market,Field,Colombian Restaurant,Gaming Cafe,Gym,0
2,M5A,-79.360636,Coffee Shop,Park,Bakery,Café,Mexican Restaurant,Pub,Breakfast Spot,Electronics Store,Farmers Market,Spa,Chocolate Shop,Italian Restaurant,Beer Store,Brewery,Dessert Shop,Ice Cream Shop,French Restaurant,Restaurant,Bank,Performing Arts Venue,Shoe Store,Event Space,Theater,Cosmetics Shop,Art Gallery,Historic Site,Antique Shop,Health Food Store,Gym / Fitness Center,Dive Bar,Drugstore,German Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,General Entertainment,Discount Store,Diner,Gift Shop,0
3,M6A,-79.464763,Clothing Store,Furniture / Home Store,Coffee Shop,Fraternity House,Miscellaneous Shop,Arts & Crafts Store,Accessories Store,Vietnamese Restaurant,Event Space,Boutique,Women's Store,Empanada Restaurant,Electronics Store,Ethiopian Restaurant,Eastern European Restaurant,Discount Store,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Dive Bar,Dim Sum Restaurant,Diner,Farmers Market,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Falafel Restaurant,Filipino Restaurant,Fast Food Restaurant,0
4,M7A,-79.389494,Coffee Shop,Diner,Gym,Park,Burrito Place,Italian Restaurant,Spa,Japanese Restaurant,Fast Food Restaurant,Seafood Restaurant,Sandwich Place,Burger Joint,Yoga Studio,Café,Sushi Restaurant,Portuguese Restaurant,Mexican Restaurant,Chinese Restaurant,Creperie,College Auditorium,Bar,Smoothie Shop,Arts & Crafts Store,Theater,Wings Joint,Hobby Shop,Vegetarian / Vegan Restaurant,Fraternity House,Doner Restaurant,Dog Run,Dive Bar,Garden Center,Discount Store,Gastropub,Gay Bar,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega,General Entertainment,0
6,M3B,-79.352188,Japanese Restaurant,Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Basketball Court,Dumpling Restaurant,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio,Dog Run,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Eastern European Restaurant,Discount Store,Dive Bar,Cupcake Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Field,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Fast Food Restaurant,Fish & Chips Shop,Filipino Restaurant,Comic Shop,0
7,M4B,-79.309937,Fast Food Restaurant,P

##### Cluster 0: Neighborhoods with lots coffeshops and restaurants

#### Cluster 1

In [72]:
c1=toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
print('cluster_1. Nr of Neighborhoods', c1.shape[0])
c1

cluster_1. Nr of Neighborhoods 7


,Postcode,longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,Cluster Labels
0,M3A,-79.329656,Fast Food Restaurant,Park,Food & Drink Shop,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio,Diner,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Eastern European Restaurant,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop,Farmers Market,Field,Colombian Restaurant,Garden,Gym / Fitness Center,Gym,Grocery Store,1
20,M6E,-79.453512,Park,Women's Store,Fast Food Restaurant,Market,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Electronics Store,Eastern European Restaurant,Diner,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Discount Store,Dessert Shop,Dim Sum Restaurant,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop,Farmers Market,Field,Hakka Restaurant,Gaming Cafe,Gym,Grocery Store,1
39,M3K,-79.464763,Park,Airport,Bus Stop,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Diner,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Discount Store,Dim Sum Restaurant,Field,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop,Fast Food Restaurant,Yoga Studio,Comfort Food Restaurant,Garden,Gym / Fitness Center,Gym,1
62,M9N,-79.518188,Park,Yoga Studio,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Diner,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Discount Store,Dim Sum Restaurant,Colombian Restaurant,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop,Fast Food Restaurant,Field,Filipino Restaurant,Garden,Gym / Fitness Center,Gym,Grocery Store,1
83,M1V,-79.284577,Park,Playground,Yoga Studio,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Dim Sum Restaurant,Department Store,Farmers Market,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop,Comfort Food Restaurant,Falafel Restaurant,Fast Food Restaurant,College Stadium,German Restaurant,Garden,Garden Center,Gastropub,1
89,M4W,-79.377529,Park,Playground,Building,Trail

##### Neighborhoods with many parks and not many restaurants. Looks like  residential neighborhoods

### Cluster 2

In [73]:
c2=toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
print('cluster_2. Nr of Neighborhoods', c2.shape[0])
c2

cluster_2. Nr of Neighborhoods 1


,Postcode,longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,Cluster Labels
44,M2L,-79.374714,Cafeteria,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Colombian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop,Fast Food Restaurant,Field,Filipino Restaurant,Garden,Gym / Fitness Center,Gym,Grocery Store,2


##### Neighborhoods with lots of restaurants of different world's countries

### Cluster 3

In [74]:
c3=toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
print('cluster_3. Nr of Neighborhoods', c3.shape[0])
c3

cluster_3. Nr of Neighborhoods 7


,Postcode,longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,Cluster Labels
9,M6B,-79.445073,Park,Sushi Restaurant,Pizza Place,Japanese Restaurant,Pub,Yoga Studio,Doner Restaurant,Discount Store,Dive Bar,Dog Run,Drugstore,Donut Shop,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Diner,Department Store,Dessert Shop,Cosmetics Shop,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Coworking Space,Falafel Restaurant,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Event Space,Fast Food Restaurant,Farmers Market,Furniture / Home Store,3
34,M4J,-79.338106,Park,Pizza Place,Coffee Shop,Convenience Store,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Dumpling Restaurant,Dim Sum Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Diner,Department Store,Dessert Shop,Farmers Market,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Construction & Landscaping,Concert Hall,Comic Shop,Comfort Food Restaurant,Falafel Restaurant,Yoga Studio,Colombian Restaurant,German Restaurant,Garden,Garden Center,Gastropub,3
61,M6N,-79.487262,Bus Line,Caribbean Restaurant,Pizza Place,Convenience Store,Yoga Studio,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Dumpling Restaurant,Diner,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Discount Store,Dessert Shop,Dim Sum Restaurant,Colombian Restaurant,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Construction & Landscaping,Concert Hall,Comic Shop,Farmers Market,Fast Food Restaurant,Field,Gift Shop,Garden Center,Gastropub,3
68,M9P,-79.532242,Pizza Place,Intersection,Sandwich Place,Coffee Shop,Chinese Restaurant,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Yoga Studio,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Drugstore,Dessert Shop,Farmers Market,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop,Falafel Restaurant,Fast Food Restaurant,Colombian Restaurant,German Restaurant,Garden,3
70,M2R,-79.442259,Pharmacy,Grocery Store,Pizza Place,Discount Store,Coffee Shop,Drugstore,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Dumpling Restaurant,Dessert Shop,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Dim Sum Restaurant,Deli / Bodega,Department Store,Farmers Market,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop,Comfort Food Restaurant,Falafel Restaurant,Fast Food Restaurant,Colle

#### Cluster 3: Places wherre venues are more heterogenous and not many restaurants

### Cluster 4

In [75]:
c4=toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
print('cluster_4. Nr of Neighborhoods', c4.shape[0])
c4

cluster_4. Nr of Neighborhoods 1


,Postcode,longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,Cluster Labels
5,M1B,-79.194353,Fast Food Restaurant,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Harbor / Marina,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Diner,Yoga Studio,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop,Comfort Food Restaurant,Dim Sum Restaurant,Field,Filipino Restaurant,Garden,Gym / Fitness Center,Gym,Grocery Store,4


#### Clauster 4: Neighborhood with many aquatic venues. The Neighborhood is by the river

### Cluster 5

In [76]:
c5=toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
print('cluster_5. Nr of Neighborhoods', c5.shape[0])
c5


cluster_5. Nr of Neighborhoods 1


,Postcode,longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,Cluster Labels
31,M1J,-79.239476,Playground,Convenience Store,Yoga Studio,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Dim Sum Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Diner,Dessert Shop,Fast Food Restaurant,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Construction & Landscaping,Concert Hall,Comic Shop,Comfort Food Restaurant,Farmers Market,Field,College Stadium,Gift Shop,Garden Center,Gastropub,Gay Bar,5


#### Cluster 5: Neighborhood where the most relevant venues are Playground

### Cluster 6

In [77]:
c6=toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
print('cluster_5. Nr of Neighborhoods', c6.shape[0])
c6


cluster_5. Nr of Neighborhoods 1


,Postcode,longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,Cluster Labels
21,M1G,-79.216917,Coffee Shop,Pharmacy,Korean Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Dim Sum Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Diner,Dessert Shop,Fast Food Restaurant,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop,Comfort Food Restaurant,Farmers Market,Field,College Stadium,Gaming Cafe,Gym,Grocery Store,6


##### Cluster 6: Places with many Pharmacoes, few restaurants and few shops